In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from ultralytics import YOLO

### Load model
We are using the nano version, which is the fastest

In [4]:
model = YOLO("yolov8n.yaml")

### Extracting and preparing stuff
For the YOLO, we need the following structure:

/dataset

    ../images
    ../labels
        ../label.txt
        
For YOLO, we need .txt files with the same name as the images and with the following structure:

class_name x_center y_center box_width box_height

Here's an example with two boxes:

0 0.7667410714285714 0.9377455357142858 0.19839285714285712 0.1245089285714285

0 0.6230133928571429 0.9749330357142857 0.17245535714285717 0.050133928571428524

__Run the following only if the dataset is not in the required formtat!__

In [ ]:
import os
import xml.etree.ElementTree as ET

In [ ]:
def extract_bbox(xml_file,output_file):
    with open(output_file, 'w') as f:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        for obj in root.findall('.//object'):
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = float(bbox.find('xmin').text)
            xmax = float(bbox.find('xmax').text)
            ymin = float(bbox.find('ymin').text)
            ymax = float(bbox.find('ymax').text)

            x_center, y_center, box_width, box_height = convert_to_yolo(xmin, xmax, ymin, ymax, 224, 224)
            line = f"{0} {x_center} {y_center} {box_width} {box_height}\n"
            f.write(line)
    os.remove(xml_file)

def convert_to_yolo(xmin, xmax, ymin, ymax, img_width, img_height):
    x_center = (xmin + xmax) / (2 * img_width)
    y_center = (ymin + ymax) / (2 * img_height)

    box_width = (xmax - xmin) / img_width
    box_height = (ymax - ymin) / img_height

    return x_center, y_center, box_width, box_height


In [ ]:
#for i in range(21):
#    extract_bbox(f'dataset2yolo/labels/sp_{i}.xml',f'dataset2yolo/labels/sp_{i}.txt')

directory = os.fsencode("dataset2_yolo/labels/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    output = filename.split('.xml')[0] + '.txt'
    extract_bbox("dataset2_yolo/labels/" + filename,"dataset2_yolo/labels/" + output)

In [ ]:
i = 0
for file in os.listdir(os.fsencode("dataset2_yolo/labels/")):
    #filename = os.fsdecode(file)
    #output = filename.split('.xml')[0] + '.txt'
    #extract_bbox("dataset_yolo/labels/" + filename,"dataset_yolo/labels/" + output)
    i+=1
print("All: " + str(i))
print("Train: " + str(2619))
print("Test: " + str(1122))

All: 101
Train: 2619
Test: 1122


### Training the model
Check config.yaml

In [5]:
yaml_path = '/content/drive/MyDrive/Deep Learning/Project/dataset_yolo/config.yaml'

In [6]:
results = model.train(data=yaml_path, epochs=100, imgsz=224)

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Deep Learning/Project/dataset_yolo/config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=

100%|██████████| 755k/755k [00:00<00:00, 24.3MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 106MB/s]


AMP: checks passed ✅


...

    100/100     0.606G      1.429      0.751      1.105         17        224: 100%|██████████| 286/286 [01:06<00:00,  4.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:08<00:00,  4.03it/s]

                   all       1145       1126       0.91       0.82      0.886      0.488



100 epochs completed in 2.463 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:13<00:00,  2.62it/s]


                   all       1145       1126      0.906      0.826      0.886      0.489
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/train


In [ ]:
model = YOLO('runs/detect/train/weights/last.pt')

In [ ]:
results = model.train(resume=True)

In [9]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [15]:
!zip -r trained_yolo.zip runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/val/ (stored 0%)
  adding: runs/detect/val/val_batch1_pred.jpg (deflated 8%)
  adding: runs/detect/val/confusion_matrix.png (deflated 40%)
  adding: runs/detect/val/val_batch2_labels.jpg (deflated 10%)
  adding: runs/detect/val/val_batch1_labels.jpg (deflated 9%)
  adding: runs/detect/val/PR_curve.png (deflated 21%)
  adding: runs/detect/val/confusion_matrix_normalized.png (deflated 39%)
  adding: runs/detect/val/val_batch0_labels.jpg (deflated 9%)
  adding: runs/detect/val/R_curve.png (deflated 18%)
  adding: runs/detect/val/val_batch2_pred.jpg (deflated 10%)
  adding: runs/detect/val/val_batch0_pred.jpg (deflated 8%)
  adding: runs/detect/val/F1_curve.png (deflated 18%)
  adding: runs/detect/val/P_curve.png (deflated 19%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 10%)
  adding: runs/detect/train/confusion_matrix.png (deflated 39%)
  adding: run

In [ ]:
print(results)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000027240858370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [13]:
model = YOLO('runs/detect/train/weights/best.pt')

In [14]:
metrics = model.val()

Ultralytics YOLOv8.2.18 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/.shortcut-targets-by-id/1c1NE_DgW7BpjHmTTBYZKbh9j2LVEQZ6I/Project/dataset_yolo/labels/val.cache... 577 images, 568 backgrounds, 0 corrupt: 100%|██████████| 1145/1145 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:19<00:00,  3.76it/s]


                   all       1145       1126      0.903      0.827      0.885       0.49
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val


In [ ]:
print(metrics)

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000272409BD580>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

TypeError: 'NoneType' object is not subscriptable

### Check stats
To check stats go to the folder run/detect/train

In [ ]:
from PIL import Image
from IPython.display import display

In [ ]:
img = Image. open('runs/detect/train22222/confusion_matrix.png')
display(img)

In [ ]:
img2 = Image. open('runs/detect/train22222/val_batch0_pred.jpg')
display(img2)

img3 = Image. open('runs/detect/train22222/val_batch0_labels.jpg')
display(img3)